In [1]:
import keras
from keras.applications import VGG19
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.optimizers import Adam

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
IMAGE_SIZE = [224, 224]

train_path = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training'
test_path = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing'

In [4]:
vgg19_model = VGG19(input_shape=IMAGE_SIZE + [3],weights='imagenet', include_top=False)

In [5]:
for layer in vgg19_model.layers:
    layer.trainable = False

In [6]:
folders = glob('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training/*')
x = Flatten()(vgg19_model.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg19_model.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
train_generator = train_datagen.flow_from_directory('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')




test_generator = test_datagen.flow_from_directory('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 4600 images belonging to 2 classes.
Found 1872 images belonging to 2 classes.


In [10]:
model = Sequential()


model.add(vgg19_model)

# Flatten the output and add a dense layer
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
epochs = 30
batch_size = 32
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


Epoch 1/30
143/143 [==============================] - 1125s 8s/step - loss: 0.5492 - accuracy: 0.8113 - val_loss: 0.1899 - val_accuracy: 0.9300
Epoch 2/30
143/143 [==============================] - 1174s 8s/step - loss: 0.2003 - accuracy: 0.9232 - val_loss: 0.1394 - val_accuracy: 0.9526
Epoch 3/30
143/143 [==============================] - 1282s 9s/step - loss: 0.2107 - accuracy: 0.9153 - val_loss: 0.1241 - val_accuracy: 0.9537
Epoch 4/30
143/143 [==============================] - 1163s 8s/step - loss: 0.1275 - accuracy: 0.9514 - val_loss: 0.0918 - val_accuracy: 0.9709
Epoch 5/30
143/143 [==============================] - 1167s 8s/step - loss: 0.1557 - accuracy: 0.9400 - val_loss: 0.1320 - val_accuracy: 0.9467
Epoch 6/30
143/143 [==============================] - 1185s 8s/step - loss: 0.1057 - accuracy: 0.9641 - val_loss: 0.0711 - val_accuracy: 0.9747
Epoch 7/30
143/143 [==============================] - 1182s 8s/step - loss: 0.1196 - accuracy: 0.9540 - val_loss: 0.0705 - val_accuracy:

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
y_true = test_generator.classes
y_pred_raw = model.predict(test_generator)
y_pred = (y_pred_raw > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate accuracy, precision, recall, and confusion matrix
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall (Sensitivity): {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

59/59 [==============================] - 303s 5s/step
Accuracy: 0.4995
Precision: 0.5407
Recall (Sensitivity): 0.5402
Confusion Matrix:
[[384 468]
 [469 551]]


In [13]:
import tensorflow as tf
from keras.models import load_model

model.save('Brain Tumour1.h5')

In [14]:
from keras.models import load_model

In [15]:
from keras import utils

In [16]:
from keras.applications.vgg19 import preprocess_input

In [17]:
import numpy as np

In [18]:
model=load_model('Brain Tumour1.h5')

In [19]:
img_path = '/Users/yash/Downloads/project /Dataset/Testing/no_tumor/image(1).jpg'  # Change this to the path of your brain image
img = utils.load_img(img_path, target_size=(224, 224))
img_array = utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [20]:
prediction = model.predict(img_array)
prediction

1/1 [==============================] - 0s 220ms/step


array([[1.3060563e-14]], dtype=float32)

In [21]:
if prediction[0][0] >= 0.5:
         print("Tumour detected")
else:
        print("Result is Normal")

Result is Normal
